In [4]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [5]:
query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%quant-ph%'))
resultPRL = query.all()
abstract_all = [x.article.abstract for x in query.all()]
session.close_all()

In [6]:
abstract_train = abstract_all
print len(abstract_train)

60594


CountVectorizer might do a cleaner job.

In [7]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

Let's make a bunch of abstracts that start in a way similar to existing abstracts, and find if they accidentally plagarize anything.

In [8]:
def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [9]:
WriteAbstract(4)

New abstract: We consider one copy of a quantum state undergoing radiative decay depends on the algebraic definition of counterfactual computation can be achieved via the exchange interaction under a diagonal form. Rewriting the kinetic term in the double slit interference of visible light with the decay of a massive body in flat space time can be unravelled into stochastic trajectories described by a weak probe field through the oscillator s Hamiltonian. For a certain value omega # this appeares to to be a physical observable is represented as a von Neumann algebra N can equivalently be described as a unitary interaction with helper indeed two variants one where the breaking of scale symmetry takes place with an algebraic structure it is not clear what role each of the fundamental aim of physical investigation.

Existing abstract: We have constructed the geometric phases emerging from the non trivial topology of a space dependent magnetic field interacting with the spin magnetic momen

<__main__.AbstractWriter instance at 0x10771dcb0>

In [10]:
WriteAbstract(5)

New abstract: Quantum measurements are considered for optimal control of quantum systems by driving a control parameter as near to the adiabatic limit as possible over the entire protocol duration. Specifically we show that the same result holds provided that we relax the tensor structure of space like separated atoms one of which is free from explicit dependence upon intrinsic fluctuations of observables. The first error disturbance relation is unfounded since it is based on an oracularizing property of certain private information retrieval systems QSPIRs that use sublinear communication do not use shared randomness among the servers and preserve privacy against honest users and dishonest servers. Classically SPIRs without shared randomness do not exist at all. Otherwise the problems of how to implement a Mach Zehnder interferometry using a quantized Bose Josephson junction with negative charging energy. The quantum adiabatic evolution through its bifurcation point. Here we propose a 

<__main__.AbstractWriter instance at 0x1236f4320>

In [11]:
WriteAbstract(6)

New abstract: A prominent example of a topologically ordered phase and typically involves four spin interactions on a two dimensional kagome lattice. By using the Stochastic Green Function algorithm we show that the generated states are equal to the total angular momentum eigenstates obtained via the usual quantum mechanical coupling of angular momenta. In this way we demonstrate quantitatively the relation between the purity negativity and a universal entanglement witness for two qubit entanglement. We also prove some simple results on structural stability within the set of all possible theories under a weak empirical topology the set of all theories with retrograde signals are both open and dense.

Existing abstract: We present a mathematical proof of the algorithm allowing to generate all symmetric and non symmetric total angular momentum eigenstates in remote matter qubits by projective measurements proposed in Maser et al. Phys. Rev. A ## ###### #### . By deriving a recursion form

<__main__.AbstractWriter instance at 0x11f469b48>